From VENUES_DIR to EXTRACTED_VENUES

In [3]:
FRSQ_OUTPUT_PATH ='../data/output/frsq/'
VENUES_DIR = FRSQ_OUTPUT_PATH+'raw_venues/'

EXTRACTED_VENUES = FRSQ_OUTPUT_PATH+'frsq_extracted_venues.geojson'
import glob
import csv
import json
from ast import literal_eval
import json
import geopandas as gp

In [5]:
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display
TARGET_KEYS = ['id', 'name', 'location', 'categories', 'verified', 'stats']
all_venues = []
import json
for fn in glob.glob(VENUES_DIR+'*.txt'):
    print fn
    with open(fn) as f:
        for line in f:
            swne, vcnt, data = line.split('\t')
            data = literal_eval(data)
            venues = data['venues']
            if venues:
                print json.dumps(venues,indent=4)
            all_venues.extend(venues)
            break

len(all_venues)

../data/output/frsq/raw_venues\10000.txt
../data/output/frsq/raw_venues\12000.txt
[
    {
        "verified": false, 
        "name": "Oki Bowl DC & Sake", 
        "allowMenuUrlEdit": true, 
        "venueChains": [], 
        "categories": [
            {
                "pluralName": "Noodle Houses", 
                "primary": true, 
                "name": "Noodle House", 
                "shortName": "Noodles", 
                "id": "4bf58dd8d48988d1d1941735", 
                "icon": {
                    "prefix": "https://ss3.4sqi.net/img/categories_v2/food/ramen_", 
                    "suffix": ".png"
                }
            }
        ], 
        "specials": {
            "count": 0, 
            "items": []
        }, 
        "url": "http://www.okibowldc.com/home/", 
        "contact": {
            "facebook": "663767173741737", 
            "phone": "2027506703", 
            "facebookUsername": "okibowldc", 
            "formattedPhone": "(202) 750-6703", 
      

18

In [5]:
df_venues = json_normalize(all_venues)

In [6]:
import shapely.geometry as shpgeo

In [7]:
df_venues['categories.name'] = df_venues.categories.apply(lambda x: x[0]['name'] if x else '')
df_venues['categories.id'] = df_venues.categories.apply(lambda x: x[0]['id'] if x else '')
df_venues['geometry'] = df_venues.apply(lambda x: shpgeo.Point(x['location.lng'], x['location.lat']), axis=1)

In [8]:
df_venues['location.isFuzzed'].value_counts()

True    2
dtype: int64

In [9]:
print df_venues['location.labeledLatLngs'].apply(lambda x: x[0]['label'] if type(x)!=float else 0).value_counts()
df_venues.apply(lambda x: (x['location.labeledLatLngs'][0]['lng']/x['location.lng'] )
                            if type(x['location.labeledLatLngs'])!=float else 0, axis=1).value_counts()

display    42793
0            211
dtype: int64


1    42793
0      211
dtype: int64

In [10]:
columns = ['id', 'geometry', 'name', 'stats.checkinsCount', 'stats.tipCount', 'stats.usersCount','categories.name']
df_no_dup = df_venues[columns].drop_duplicates('id').copy()
df_no_dup.columns = ['id', 'geometry', 'name', 'checkins', 'tips', 'users','category']
df_no_dup.shape

(34731, 7)

In [14]:
dc_poly_path = '../data/dc_polygon.geojson'
dc_poly_gpdf = gp.read_file(dc_poly_path)
dc_poly = dc_poly_gpdf.geometry.values[0]

In [16]:
print df_no_dup.geometry.apply(lambda x: x.intersects(dc_poly)).value_counts()
df_no_dup_in_dc = df_no_dup[df_no_dup.geometry.apply(lambda x: x.intersects(dc_poly))].copy()

True     34060
False      671
dtype: int64


In [17]:
import geopandas as gp
with open(EXTRACTED_VENUES, 'w') as f:
    f.write(gp.GeoDataFrame(df_no_dup_in_dc).to_json())